<div style="display: flex;">
    <span style="margin-left: auto; margin-right: 5px;  margin-top: 1px;"><a href="https://creativecommons.org/licenses/by/4.0/">
        <img src="https://licensebuttons.net/l/by/4.0/80x15.png" />
    </a></span>
    <span style="margin-right: auto; margin-left: 5px;"><a href="https://opensource.org/licenses/MIT">
        <img src="https://img.shields.io/badge/License-MIT-green.svg" />
    </a></span>
</div>

# pyBarSim in 2.5D

**Author:** Guillaume Rongier

In this notebook, we will look at simulating the deposits of wave-dominated shallow-marine environments in 2.5D using pyBarSim.

### Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cmocean
import pyvista as pv

from pybarsim import BarSimPseudo3D
from pybarsim.barsim import prepare_subsequence_grid

## 1. Setup and run

Define the initial elevation and cell size (in m):

In [ ]:
initial_elevation = np.linspace(np.linspace(1000., 900., 200), np.linspace(1000., 850., 200), 150)

spacing = (100., 100.)

Define the run time (in yr) and the inflection points for the variations of sea level (in m):

In [ ]:
run_time = 25000.

sea_level_curve = np.array([
    (0., 998.),
    (0.25*run_time, 985.),
    (0.5*run_time, 975.),
    (0.75*run_time, 985.),
    (run_time + 20., 998.)
])

Define the inflection points for the sediment supply, which varies along-shore (in m$^2$/yr):

In [ ]:
sediment_supply_curve = np.array([
    np.tile([[0.], [0.25*run_time], [0.5*run_time], [0.75*run_time], [run_time + 20.]], 200),
    np.vstack([
        np.linspace(25., 5., 200),
        np.linspace(25., 5., 200),
        np.linspace(25., 5., 200),
        np.linspace(5., 1., 200),
        np.linspace(5., 1., 200),
    ])
]).T

Initialize a `BarSimPseudo3D` object and run the simulation:

<div class="alert alert-block alert-warning">
<b>&#9888;</b> This takes more time to run the first time because Numba needs to compile the Python code.
</div>

In [ ]:
barsim = BarSimPseudo3D(initial_elevation,
                        sea_level_curve,
                        sediment_supply_curve,
                        spacing=spacing,
                        max_wave_height_fair_weather=1.5,
                        allow_storms=True,
                        start_with_storm=False,
                        max_wave_height_storm=6.,
                        tidal_amplitude=2.,
                        min_tidal_area_for_transport=100.,
                        sediment_size=(5., 50., 125., 250.),
                        sediment_fraction=(0.25, 0.25, 0.25, 0.25),
                        initial_substratum=(200., (0.25, 0.25, 0.25, 0.25)),
                        erodibility=0.1,
                        washover_fraction=0.5,
                        tide_sand_fraction=0.3,
                        depth_factor_backbarrier=5.,
                        depth_factor_shoreface=10.,
                        local_factor_shoreface=1.5,
                        local_factor_backbarrier=1.,
                        fallout_rate_backbarrier=0.,
                        fallout_rate_shoreface=0.0002,
                        max_width_backbarrier=500.,
                        curve_preinterpolation=None,
                        seed=42)
barsim.run(run_time, dt_fair_weather=15., dt_storm=1.)

Similarly to the 2D case, we can reduce the number of time steps to make plotting easier:

In [ ]:
barsim.finalize(on='sequence')

In [ ]:
barsim.subsample(20)

In [ ]:
barsim.finalize(on='subsequence')

We can also reinterpolates the result to a regular grid:

In [ ]:
barsim.regrid(850., 1000., 0.5)

In [ ]:
barsim.finalize(on='record')

## 2. Stratigraphy visualization

The same functions used in 2D for plotting `subsequence_` can be used in 2.5D, first to extract a section along the x axis (at index 75 along the y axis):

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))
ax.fill_between(barsim.subsequence_['X'][:-1],
                barsim.subsequence_['Horizons'][0, 75, :-1],
                barsim.subsequence_['Horizons'][0, 75, :-1].min(),
                color='#d9d9d9')
c = barsim.plot_subsequence(ax, 75, var='Mean grain size')
fig.colorbar(c[0], ax=ax, label=r'Mean grain size ($\mu$m)')
ax.set(xlabel='x (m)', ylabel='z (m)');

And we can compute the water depth:

In [ ]:
water_depth = barsim.sequence_['Sea level'] - barsim.sequence_['Elevation']
barsim.sequence_['Water depth'] = water_depth.where(water_depth > 0., 0.)

To plot it along a well extracted at index 50 along the x axis and 75 along the y axis:

In [ ]:
fig, ax = plt.subplots(figsize=(2.5, 7))
p, sm = barsim.plot_well(ax, 50, 75, on='sequence', var='Water depth', cmap=cmocean.cm.deep)
fig.colorbar(sm, ax=ax, pad=0.1, label='Water depth (m)')
ax.set(xlabel=r'Mean grain size ($\mu$m)', ylabel='z (m)');

We can directly visualize sections through `record_`, the regular grid, using [xarray's plotting functions](https://docs.xarray.dev/en/stable/user-guide/plotting.html?highlight=plotting):

In [ ]:
barsim.record_['Mean grain size'][-10].plot(figsize=(8, 6))

In [ ]:
barsim.record_['Mean grain size'][:, 75].plot(figsize=(12, 5))

In [ ]:
barsim.record_['Deposits'][0, :, 75].plot(figsize=(12, 5))

## 3. 3D visualization

We first need to extract the coordinates of the grid nodes and the property to plot from `subsequence_`:

In [ ]:
x, y, z, layers = prepare_subsequence_grid(barsim.subsequence_,
                                           var='Mean grain size')

Then we can create a [PyVista](https://docs.pyvista.org/) mesh:

In [ ]:
mesh = pv.StructuredGrid(x, y, z)

Add the property to plot:

In [ ]:
mesh['Mean grain size'] = layers.ravel()

And plot the grid:

In [ ]:
p = pv.Plotter()
p.add_mesh(mesh, scalars='Mean grain size', lighting=False)
p.set_scale(zscale=50)
p.show()

From there we can use any of PyVista's filters, for instance to view slices along the x and y axes:

In [ ]:
p = pv.Plotter()
p.add_mesh(mesh.slice_along_axis(n=10, axis='x'), scalars='Mean grain size', lighting=False)
p.add_mesh(mesh.slice_along_axis(n=10, axis='y'), scalars='Mean grain size', lighting=False)
p.set_scale(zscale=50)
p.show()

Visualizing `record_` is more direct. We can directly create the mesh:

In [ ]:
mesh = pv.ImageData(dimensions=(201, 151, 301),
                    spacing=(100., 100., 0.5,),
                    origin=(0., 0., 850.))

And add the property to plot, removing the lower and upper part of the grid to only plot BarSim's deposits:

In [ ]:
values = barsim.record_['Major facies'].values.astype(float)
values[(barsim.record_['Facies'][0] > 0) | (barsim.record_['Facies'][1] > 0)] = np.nan
mesh['Major facies'] = values.ravel()

We can then visualize the deposits in the regular grid:

In [ ]:
p = pv.Plotter()
p.add_mesh(mesh.threshold(), scalars='Major facies', lighting=False)
p.set_scale(zscale=50)
p.show()